# Model 2

- 목적 : model_1을 통과하여 imp로 처리된 데이터를 다시 분류한다.
- 학습 데이터 : dktc_imp, 4개의 레이블이 달린 기타 imp 대화셋 추가 가능.
- 참고사항 : immoral에서 수동으로 imp 추출해볼까도 생각중이지만... ... ... ... 

In [ ]:
# !pip install keras-bert
# !pip install keras-radam
# !pip install wandb

In [1]:
# 라이브러리 import 및 주요 라이브러리 버전 확인 
import wandb
from wandb.keras import WandbCallback

import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle
from sklearn.model_selection import train_test_split

import keras as keras
from keras.models import load_model
from keras import backend as K 
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

import json

import os
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.logging.set_verbosity(tf.logging.ERROR)

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

In [ ]:
import encoding_korbert as enc

In [6]:
dktc_imp_df = pd.read_csv("./data/dktc_imp.csv", usecols=['class','conversation'])

In [7]:
for i in range(10):
    dktc_imp_df = dktc_imp_df.sample(frac=1).reset_index(drop=True) #많이 섞기

In [9]:
dktc_imp_df["class"] = dktc_imp_df["class"].astype('category')
dktc_imp_df["class"] = dktc_imp_df["class"].cat.codes

In [10]:
dktc_imp_df
# 0: 갈취 대화, 1: 기타 괴롭힘 대화 , 2: 직장 내 괴롭힘 대화, 3: 협박 대화

,class,conversation
0,0,야 거기 찐따. 일로와봐.\n저요.?\n여기에 찐따가 너 말고 누가 있어.\n왜요....
1,3,자네집 개가 우리집 개를 물어서 피가 나고 살점이 뜯겨서 병원까지 다녀왔네\n아 미...
2,1,이거봐봐 우리반 길동이 지 올린건데 뭐냐 이거 \n 완전 표정 왜저래 지가 이쁜줄 ...
3,1,내가 쟤랑 왜 같은 조냐고\n야 좀 참아 어차피 쟤가 다 해줄걸?\n그런가? 하긴....
4,1,아가씨 우리랑 같이 놀까?\n제가 막차가 얼마 안 남아서요. 죄송합니다.\n에이 왜...
...,...,...
3945,3,내일까지 해결하는 게 좋을거야\n불가능합니다. 시간이 너무 촉박해요\n촉박이라.\n...
3946,2,김과장 이리와보게\n부장님 무슨일로 부르셨습니까??\n내가 하라고 한 프로젝트를 이...
3947,2,모레 회사 단합회에서 장기자랑 있는 거 알고있지?\n네 과장님.\n그래서 말인데 난...
3948,0,야 거기 너 일로 와바\n네? 저요?\n너 말고 누구겠냐\n저 아무것도 없어요.\n...


### train_validation_split

In [11]:
model_2_train_df, model_2_val_df = train_test_split(dktc_imp_df, 
                                  test_size=0.2, 
                                  random_state=5555)

In [12]:
model_2_train_df.reset_index(drop="index", inplace=True)
model_2_val_df.reset_index(drop="index", inplace=True)

### korBERT 불러와서 model_extra 빌딩& 파인튜닝

In [ ]:
tokenizer = enc.tokenizer

In [ ]:
train_x, train_y = enc.load_data(model_2_train_df)
val_x, val_y = enc.load_data(model_2_val_df)

### WandB 로그인

In [ ]:
wandb.login(key="fff4dcdf86063b9dafa0296b4abaeb7d3639da7d")

### init_config

In [ ]:
default_config = {
#     "n_channel_1" : 32,
#     "n_channel_2" : 64,
#     "n_dense" : 4,
    "learning_rate" : 5e-5,
    "epochs" : 5,
    "batch_size" : 8,
    "weight_decay" : 0.025,
    "optimizer" : "radam",
    "loss" : "categorical_crossentropy",
#     "loss" : "sparse_categorical_crossentropy",
    "metrics" : ["accuracy"]
}

### sweep_config

In [ ]:
sweep_config = {
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "optimizer" : {
            'value' : 'radam'
            },
        "batch_size" : {
            "values" : [8, 16] # OOM 에러시 바꿔주세용
            },
        "learning_rate" : {
            "min" : 5e-5,
            "max" : 5e-4 # 0에 가까울 수록
            },
        "weight_decay" : {
            "values" : [0.025, 0.001]
        },
        "epochs" : {
            "distribution" : "int_uniform",
            "min" : 3,
            "max" : 5
            }
                    
        }
    }

In [ ]:
pretrained_path ="./bert" #상대경로 잡기

SEQ_LEN = enc.SEQ_LEN

config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')

In [ ]:
def build_model():
    model = load_trained_model_from_checkpoint(config_path,
                                                checkpoint_path,
                                                training=True,
                                                trainable=True,
                                                seq_len=SEQ_LEN)
    
    inputs = model.inputs[:2]
    dense = model.layers[-3].output
    
    outputs = keras.layers.Dense(4, activation='softmax', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)

    bert_model = keras.models.Model(inputs, outputs)

    return bert_model

In [ ]:
# def set_optimizer():
#     if config.optimizer == "radam":
#         return RAdamOptimizer(learning_rate=config.learning_rate, 
#                               weight_decay=config.weight_decay)

In [ ]:
CLASS_NAMES = ["갈취 대화", "기타 괴롭힘 대화" , "직장 내 괴롭힘 대화", "협박 대화"]

In [ ]:
path= './saved'

layer_num = 12

In [ ]:
def keras_callbacks():                       # 날짜 바꿔주세용
    CK = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(path, "model_2_weights_0419.{epoch:02d}-{val_loss:.2f}.h5"),
                                              monitor='val_loss',
                                              save_best_only = True,
                                              save_weights_only = True)

    ES = tf.keras.callbacks.EarlyStopping(patience=1)
    
    return CK, ES

In [ ]:
def train():
    global default_config

    # wandb.init & config ok
    run = wandb.init(project = 'model_2_0419', # 날짜만 변경해주세요
                    entity = "m05",
                    config = default_config)
    config = wandb.config

    # Model ok
    model = build_model()    
    
    # optimizer 함수 불러오기 ok
#     optimizer = set_optimizer()
    if config.optimizer == "radam":
        OPTIMIZER = RAdamOptimizer(learning_rate=config.learning_rate, 
                              weight_decay=config.weight_decay)
    
    # model.compile ok
    model.compile(optimizer = OPTIMIZER,
                  loss = config.loss,
                  metrics= config.metrics)
    
    #keras_callbacks ok
    CK, ES = keras_callbacks()
    
    # one_hot encoding
    train_y = tf.one_hot(train_y, depth=4)
    val_y = tf.one_hot(val_y, depth=4)
    
    # model.fit
    model.fit(train_x, train_y,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data = (val_x, val_y),
              callbacks = [CK, ES , 
                           WandbCallback(training_data = (train_x[:30], train_y[:30]),
                                         validation_data = (val_x[:30], val_y[:30]),
                                        labels = CLASS_NAMES)])
                                        
    # weight 모델 저장
    tf.keras.models.save_model(model, 'model_2')
    
    # validation Accuracy[Loss] 입력하기
    wandb.log({"Val_Accuracy ": model.history.history["val_accuracy"][-1], 
               "Val_Loss ": model.history.history["val_loss"][-1]})
    
#     # evaluate test
#     test_loss, test_accuracy = model.evaluate(test_x, test_y, verbose=2)
    
#     # wandb's 테이블쪽 column 추가하기
#     wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
#                "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})
    
    run.finish() # run 종료

### model.fit 실행

In [ ]:
sweep_id = wandb.sweep(sweep_config,
                       entity = 'm05', 
                       project = 'model_2_0419') # 날짜만 변경해주세요

# run the sweep
wandb.agent(sweep_id,
            count=5, # 몇회 돌릴지 선택해주세요
            function=train)

---
---

In [42]:
model_2 = build_model()
model_2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
# for layer in model_2.layers[:layer_num]:
#     layer.trainable = False # 층을 선택하여 freeze 하기.